# Robot control Lab 6 - OpenCV poses and stereo

Todays lab will focus on extracting 3D information from images. During creation of this scenario Google Colab runs OpenCV version [4.1.2](https://docs.opencv.org/4.1.2/index.html), if you have a different version you can change docs version to match. If you want more materials or different approach to what will be presented in this scenario, [these materials](https://docs.opencv.org/4.1.2/d9/db7/tutorial_py_table_of_contents_calib3d.html) should talk about similar things.

## Working with Colab - reminder

As a reminder you should make a copy of this Colab notebook so that you can save changes you make and save your code for later. You can use your [UW Google account](https://it.uw.edu.pl/pl/uslugi/UslugiMojaPocztaGmailStudent/).

Go through the whole notebook by expanding subsequent sections.

## Downloading assets, imports



Execute cell below to download assets we will be using today. If you have not launched the runtime yet it will probably launch automatically now.

In [ ]:
%cd /content/
!rm -rf lab6_assets
!git clone https://github.com/nomagiclab/lab6_assets.git
%cd lab6_assets

import cv2
import numpy as np
print(f"OpenCV version is: {cv2.__version__}")

# In Colab we need to use:
from google.colab.patches import cv2_imshow

## Calibration parameters, rectification


Today we will be working with a camera with given calibration parameters, this means someone else (maybe manufacturer) performed the calibration procedure and supplied us with the results. As you probably remember from the lecture, the camera matrix, or calibration matrix looks like so:

\begin{align}
\left[\begin{array}{ccc}
f_{x} & 0 & c_{x}\\
0 & f_{y} & c_{y}\\
0 & 0 & 1
\end{array}\right]
\end{align}

And we talked about radial and tangential distortions, we represent them with a vector of 5 numbers:

\begin{align}
\left(\begin{array}{ccccc}
k_{1} & k_{2} & p_{1} & p_{2} & k_{3}\end{array}\right)
\end{align}

More on that [here](https://docs.opencv.org/4.1.2/dc/dbb/tutorial_py_calibration.html).

In [ ]:
camera_matrix = np.array([[528.86 ,   0.    , 641.865],
                          [  0.   , 528.755 , 360.867],
                          [  0.   ,   0.    ,   1.   ]])

dist_coeffs = np.array([-0.0420881, 0.0110919, -0.00090298, -0.00013151, -0.00534522])

Today asset pictures are **not** rectified. We have calibration parameters, so this should not be a problem. Let's say we just want to **rectify** the image for now. We will derive a new camera matrix using [getOptimalNewCameraMatrix](https://docs.opencv.org/4.1.2/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1), try to set alpha to 0 and 1 and see what is the difference. Then we will calculate rectification maps. Notice that we do not have to do this with every frame, once calculated, these maps can be used to undistort multiple pictures (most likely subsequent frames from the camera).

Below code is mostly a demo of how to undistort, but you need to try alpha 0 and 1 on your own!

In [ ]:
img0 = cv2.imread("0.png")

# OpenCV order
size = (img0.shape[1], img0.shape[0])

# Calculate rectified camera matrix
alpha = 0 # TODO: try 0 and 1
rect_camera_matrix = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, size, alpha)[0]

# Calculate rectification maps
map1, map2 = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, np.eye(3), rect_camera_matrix, size, cv2.CV_32FC1)

# Use maps to rectify an image
rect_img0 = cv2.remap(img0, map1, map2, cv2.INTER_LINEAR)

# Show original and rectified side by side
cv2_imshow(cv2.hconcat([img0, rect_img0]))

## Single image, single marker pose

### Quick recap, OpenCV names:

Pose = postition + orientation

Position is a 3D translation vector:
$\left[\begin{array}{c}d_{x}\\d_{y}\\d_{z}\end{array}\right]$, OpenCV will call this `tvec`.

Orientation can be expressed in many ways: rotation matrix, rotation vector, quaternion, euler angles (e.g. roll, pitch, yaw).

OpenCV most often works with rotation vector
$\left[\begin{array}{c}r_{x}\\r_{y}\\r_{z}\end{array}\right]$,
and calls it `rvec`.

OpenCV also provides [Rodrigues](https://docs.opencv.org/4.1.2/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac) to convert `rvec` to and from the rotation matrix.

### Detecting the pose

As you remember from the lecture using pinhole camera model we can map a point in 3D world coordinates to a point on the image plane. The other direction is not so easy — point on the image plane gives us a line (light ray) in 3D on which this point in real world was. Now, we can use 4 corners of a marker, because we know their geometry, i.e. they lie on a single plane and form a square with a known side length.
The process looks like so: we take corners on the image and imagine 4 rays, then we fit a square in 3D so that each corner lies on a ray and side lengths match. This image can help you visualize the process:

![Projecting points into 3D](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99B415455D9C2AE71F)
[Image source](https://ballentain.tistory.com/40)

We will not need to worry about the implementation. OpenCV has a function for us — [estimatePoseSingleMarkers](https://docs.opencv.org/4.1.2/d9/d6a/group__aruco.html#ga84dd2e88f3e8c3255eb78e0f79571bd1).

Let's see how to use it:

In [ ]:
# Aruco detector parameters
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_APRILTAG_16h5)
detectorParams = cv2.aruco.DetectorParameters_create()
detectorParams.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_CONTOUR

# Note that unit is not specified, we just need to stick to one (here meters)
MARKER_SIDE = 0.168

img1 = cv2.imread("1.png")
img1_draw = img1.copy()

corners, ids, _ = cv2.aruco.detectMarkers(img1, dictionary, None, None, detectorParams)

# TODO: inspect the image and draw detection
cv2.aruco.drawDetectedMarkers(img1_draw, corners)
cv2_imshow(img1_draw)

# This takes multiple corners and calculates 3D pose
rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, MARKER_SIDE, camera_matrix, dist_coeffs)

# TODO: inspect type and shape of rvecs and tvecs
print(rvecs)
print(tvecs)

# We can draw a pose using OpenCV
img1_draw = img1.copy()
cv2.aruco.drawAxis(img1_draw, camera_matrix, dist_coeffs, rvecs[0], tvecs[0], 0.1)
cv2_imshow(img1_draw)

### Detections and rectification

Notice that we did not rectify the image. Marker detection often runs just fine on distorted images, but we had to pass not only the camera matrix, but also distortion coeffs to every function that calculated things in 3D. This way we didn't have to explicitly rectify whole image, which can be slow.

### Your turn, but rectify first

Now try to merge knowledge about rectification and marker's pose detection and load image 2.png, rectify it, and then perform markers detection and pose calculation.

**Do not recalculate** things we already have: rectification maps, rectified camera matrix, marker detector parameters, marker size.

Note: remember that rectification changes camera matrix and makes distortion coeffitiens zero, so you should use `rect_camera_matrix` after rectification and you can just pass `0` as distCoeffs.



In [ ]:
# TODO:
img2 = cv2.imread("2.png")
rect_img2 = cv2.remap(img2, map1, map2, cv2.INTER_LINEAR)
rect_img2_draw = rect_img2.copy()

corners, ids, _ = cv2.aruco.detectMarkers(rect_img2, dictionary, None, None, detectorParams)

cv2.aruco.drawDetectedMarkers(rect_img2_draw, corners)
cv2_imshow(rect_img2_draw)

# This takes multiple corners and calculates 3D pose
rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, MARKER_SIDE, rect_camera_matrix, 0)

rect_img2_draw = rect_img2.copy()
for rvec, tvec in zip(rvecs, tvecs):
    cv2.aruco.drawAxis(rect_img2_draw, camera_matrix, dist_coeffs, rvec, tvec, 0.1)
cv2_imshow(rect_img2_draw)

### Drawing

Just as in the last scenario we will draw 3D poses ourselves, but now in 3D!

We will use [projectPoints](https://docs.opencv.org/4.1.2/d9/d0c/group__calib3d.html#ga1019495a2c8d1743ed5cc23fa0daff8c) function, which takes object pose as `rvec` and `tvec` and an array of object points. Object points are just additional translations in object's frame of reference. For example, if we have object point `[42, 0, 0]` and will call `projectPoints` with some `rvec` and `tvec`, then the function will chain transormations from camera frame of reference first by `tvec`, then `rvec`, then translate in object's frame of reference by 42 in X direction and finally project resulting point to the image plane coordinates. For convenience the function can take multiple object points at once. Inspect the demo below:

In [ ]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
DARK_GREEN = (0, 127, 0)
RED = (0, 0, 255)
BLUE = (255, 0, 0)
VIOLET = (255, 0, 255)
CYAN = (255, 255, 0)
YELLOW = (0, 255, 255)
ORANGE = (0, 100, 255)


objpts = np.array([[0, 0, 0], [-0.5, 0, 0], [0.25, 0.25, 0]]) * MARKER_SIDE

imgpts = np.rint(cv2.projectPoints(objpts, rvecs[0], tvecs[0], rect_camera_matrix, 0,)[0]).astype(int)
imgpts = imgpts.reshape((-1, 2))

rect_img2_draw = rect_img2.copy()

cv2.circle(rect_img2_draw, (imgpts[0][0], imgpts[0][1]), 5, GREEN, -1);
cv2.circle(rect_img2_draw, (imgpts[1][0], imgpts[1][1]), 5, VIOLET, -1);
cv2.circle(rect_img2_draw, (imgpts[2][0], imgpts[2][1]), 5, ORANGE, -1);

cv2_imshow(rect_img2_draw)

### Your turn! Drawing cubes

On the `rect_img2` try to calculate and draw points in a cube pattern, like so:

![cubes](https://github.com/nomagiclab/lab6_assets/raw/master/cubes.png)


You will definitely need an array of object points that designate 8 cube corners, `rvecs` and `tvecs` (use calculated ones), `projectPoints` function and some drawing functions.

You might be interested in `drawContours` function (search the docs!), but note that it takes list of contours and a contour is a list of points. This function lets you fill the contour, like the blue square in the example above or draw only the outline, like the red square in the example above.

Remember to pass coordines of `int` type to OpenCV drawing functions. Also, some of them, like `line`, expect coordinates as a tuples, not lists or arrays (`drawContours` does not care).

In [ ]:
objpts = np.array([[-0.5, -0.5, 0], [-0.5, 0.5, 0], [0.5, 0.5, 0], [0.5, -0.5, 0],
                   [-0.5, -0.5, 1], [-0.5, 0.5, 1], [0.5, 0.5, 1], [0.5, -0.5, 1]])
objpts *= MARKER_SIDE

rect_img2_draw = rect_img2.copy()

for rvec, tvec in zip(rvecs, tvecs):
    imgpts = np.rint(cv2.projectPoints(objpts, rvec, tvec, rect_camera_matrix, 0,)[0]).astype(int)
    imgpts = imgpts.reshape((-1, 2))

    back_face = np.array([imgpts[:4]])
    front_face = np.array([imgpts[4:]])
    sides_edges = np.array([[imgpts[i], imgpts[i+4]] for i in range(4)])

    cv2.drawContours(rect_img2_draw, back_face, -1, BLUE, cv2.FILLED)
    cv2.drawContours(rect_img2_draw, sides_edges, -1, GREEN, 2)
    cv2.drawContours(rect_img2_draw, front_face, -1, RED, 2)

cv2_imshow(rect_img2_draw)

## Stereo

Now we have two cameras in one package! 

![Zed 2](https://cdn.stereolabs.com/assets/images/zed-2/zed-2-front.jpg)

Actually, we were using this camera all the time, but we were using only one (left) lens. Now it is time to use both to calculate 3D coordinates, this is called stereoscopic imaging, or stereo camera. First, lets take a look at images from both lenses. Do you see the difference?

In [ ]:
img3_left = cv2.imread("3_left.png")
img3_right = cv2.imread("3_right.png")
cv2_imshow(cv2.hconcat([img3_left, img3_right]))


Stereoscopic imaging allows us to calculate 3D coordinates without knowing aything about geometry of the object, or scene. We can even do this with a single point (which will be demonstrated soon). Stereo camera is an example of depth camera, because it can provide information about the depth of the scene.



### Depth calculation

![](https://docs.opencv.org/4.1.2/stereo_depth.jpg)

[Image source](https://docs.opencv.org/4.1.2/dd/d53/tutorial_py_depthmap.html)

Quick explanation:
 - Cameras are O and O', points on the diagram denote pinholes' locations
 - Image plane in our model can equivalently be in front of the pinhole
 - Baseline is the distance between cameras
 - Vectors f are marking direction the cameras are looking at, their length is the focal length
 - X is the point in 3D
 - x and x' mark pixel coordinates of the point as observed on the image planes
 - Z marks the depth of point X (distance from pinholes along optical axes)

Now, from similar triangles:
\begin{align}
disparity=x-x\prime=\frac{Bf}{Z}
\end{align}

Since cameras are arranged horizontally next to each other, disparity will be the difference in pixels along X axis of the image (u coordinate).



### Stereo calibration parameters

Firstly, the simple thing — because we have 2 cameras we will have separate camera matrix and distortion coefficients for each of them, this is what we called intrinsic parameters, they are applied to a specific lens + sensor combo. But this was the easy part. You might think that we only have the baseline to worry about, but in reality stereo cameras are never aligned perfectly and transformation from one pinhole to another is not just the baseline translation. So we actually have full transformation of 3D translation + rotation from the left camera to the right one. To correct for this we use a function `stereoRectify`, which will produce apropriate rectification parameters so that after such rectifications images will be aligned and ready to calculate disparity. To learn more about specific matrices read the documentation of [stereoRectify](https://docs.opencv.org/4.1.2/d9/d0c/group__calib3d.html#ga617b1685d4059c6040827800e72ad2b6).

### Demo disparity

Below you will find a demo of how to handle stereo images to calculate 3D coordinates of a point. The point we will use will be the index corner of a marker with id == 23.


In [ ]:
import calib
params = calib.load_params("SN29482946_HD.yaml")

# Calculate undistort maps according to matrices calculated earlier
map1L, map2L = cv2.initUndistortRectifyMap(params.slCalib.cameraMatrixL, params.slCalib.distCoeffsL,
                                           params.R1, params.P1, params.slCalib.size, cv2.CV_32FC1)
map1R, map2R = cv2.initUndistortRectifyMap(params.slCalib.cameraMatrixR, params.slCalib.distCoeffsR,
                                           params.R2, params.P2, params.slCalib.size, cv2.CV_32FC1)

# Undistort both images at the same time (you can use this function later)
def rectify2(left, right):
  rect_left = cv2.remap(left, map1L, map2L, cv2.INTER_LINEAR)
  rect_right = cv2.remap(right, map1R, map2R, cv2.INTER_LINEAR)
  return rect_left, rect_right

rect_img3_left, rect_img3_right = rectify2(img3_left, img3_right)

# Uncomment to see intermediate effect
# cv2_imshow(cv2.hconcat([rect_img3_left, rect_img3_right]))

# Detect on both images (in C++ you would do std::async)
corners_left, ids_left, _ = cv2.aruco.detectMarkers(rect_img3_left, dictionary, None, None, detectorParams)
corners_right, ids_right, _ = cv2.aruco.detectMarkers(rect_img3_right, dictionary, None, None, detectorParams)

rect_img3_left_draw = rect_img3_left.copy()
rect_img3_right_draw = rect_img3_right.copy()

MARKER_ID = 23

# Assumes marker MARKER_ID is visible

# Find u and v coords on the left image
for (marker_id, marker_corners) in zip(ids_left.flatten(), corners_left):
  if marker_id == MARKER_ID:
    uv_left = marker_corners[0][0]
    break

# Find u and v coords on the right image
for (marker_id, marker_corners) in zip(ids_right.flatten(), corners_right):
  if marker_id == MARKER_ID:
    uv_right = marker_corners[0][0]
    break

cv2.circle(rect_img3_left_draw, (round(uv_left[0]), round(uv_left[1])), 15, ORANGE, -1)
cv2.circle(rect_img3_right_draw, (round(uv_right[0]), round(uv_right[1])), 15, ORANGE, -1)
# Uncomment to see intermediate effect
# cv2_imshow(cv2.hconcat([rect_img3_left_draw, rect_img3_right_draw]))

# ***** The calculations *****

# Disparity is just difference of horizontal (u) coordinate
disparity = uv_right[0] - uv_left[0]

# Homogenous calculations using matrix Q from stereoRectify
homgPt = np.dot(params.Q, [uv_left[0], uv_left[1], disparity, 1.])

# Normalize to ensure last element is indeed a 1
homgPt /= homgPt[3]

# Translation is just the resulting point, without the 1
tvec = homgPt[:3]

# ***** The calculations end *****
# Reproject 3D back to image plane to verify the result
imgpts = np.rint(cv2.projectPoints(np.zeros((1, 3)), np.zeros(3), np.array(tvec), params.P1[:, :-1], 0)[0]).astype(int)
imgpts = imgpts.reshape((-1, 2))

# Note that reprojection was done with P1, so we get a point in the left image
cv2.circle(rect_img3_left_draw, (imgpts[0][0], imgpts[0][1]), 8, BLUE, -1);
cv2_imshow(cv2.hconcat([rect_img3_left_draw, rect_img3_right_draw]))

### Your turn - triangle



Modify the demo from the above to take marker id 18, resolve its corners' 3D positions, calculate 3D coordinates of a special orange triangle and draw this very special triangle. You can also check your values with a `check_triangle` function available below, it takes 3D coordinates of corners in the order from marker origin and going clockwise.

The traingle is defined like so:

![](https://github.com/nomagiclab/lab6_assets/raw/master/tri_def.png)

And this is how it should look like:

![](https://github.com/nomagiclab/lab6_assets/raw/master/tri.png)

In [ ]:
rect_img3_left_draw = rect_img3_left.copy()
rect_img3_right_draw = rect_img3_right.copy()

MARKER_ID = 18

# TODO:

# Find u and v coords on the left image
for (marker_id, marker_corners) in zip(ids_left.flatten(), corners_left):
  if marker_id == MARKER_ID:
    marker_corners_left = marker_corners[0]
    break

# Find u and v coords on the right image
for (marker_id, marker_corners) in zip(ids_right.flatten(), corners_right):
  if marker_id == MARKER_ID:
    marker_corners_right = marker_corners[0]
    break

# ***** The calculations *****

disparities = [uv_right[0] - uv_left[0] for uv_right, uv_left in zip(marker_corners_right, marker_corners_left)]
homgPts = [np.dot(params.Q, [uv_left[0], uv_left[1], disparity, 1.]) for disparity, uv_left in zip(disparities, marker_corners_left)]
homgPts = [homgPt / homgPt[3] for homgPt in homgPts]
tvecs = [homgPt[:3] for homgPt in homgPts]

up = tvecs[1] - tvecs[0]
triangle_up = tvecs[0] + 2*up
triangle_right = tvecs[3] + 2*up

triangle_tvecs = [tvecs[0], triangle_up, triangle_right]

# ***** The calculations end *****

def get_imgpt(tvec):
    return np.rint(cv2.projectPoints(np.zeros((1, 3)), np.zeros(3), tvec, params.P1[:, :-1], 0)[0]).astype(int).reshape(2)

imgpts = [get_imgpt(tvec) for tvec in triangle_tvecs]

cv2.drawContours(rect_img3_left_draw, np.array([imgpts]), -1, ORANGE, cv2.FILLED);
cv2_imshow(rect_img3_left_draw)

In [ ]:
good_answer = np.array([[156.4268906 ,  -1.83889411, 907.07904896],
                        [ 130.98910429, -276.36748328, 1091.5483355 ],
                        [ 264.68937166, -232.08875541, 1179.35253107]])

def check_answer(answer):
  if (answer - good_answer < 1e-6).all():
    print("Congratulations!")
  else:
    print("Wrong answer, try again!")

# TODO:
check_answer(triangle_tvecs)

## Dense stereo matching



Last thing that should be mentioned when talking about stereo should be dense stereo matching, i.e. we will try to find the disparity not only for some known features like a corner of a marker, but for every point in the image. This is done by trying to match pixels or blocks of pixels from the left image in the right image at the same height, but shifted left.

There are some caveats here. Matching algorithms are not always perfect and almost always require a lot of tuning and are very sensitive to parameters. As you can see below the quality of the depth map is not impressive (it is not tuned very well). Nevertheless some output is produced and we can observe that closest parts of the image are brighter than far ones. We can also calculate the depth from the disparity as demonstrated for pixel on marker 18. You can verify if depths match with what we had previously. You can also try to change StereoBM parameters to maybe improve the disparity map.

Note: here we are using pyplot imshow, because the range of disparity values is not simple 0-255 intensity value like in rgb or gray.

In [ ]:
stereo = cv2.StereoBM_create(numDisparities=9*16, blockSize = 17)

gray_left = cv2.cvtColor(rect_img3_left, cv2.COLOR_BGR2GRAY)
gray_right = cv2.cvtColor(rect_img3_right, cv2.COLOR_BGR2GRAY)
disparity_image = stereo.compute(gray_left, gray_right)

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16, 9))
plt.imshow(disparity_image, 'gray')
plt.show()

# Simple z calculation from the baseline, disparity and focal length
def calc_z(disp) -> float:
  if disp <= 0:
    return 0
  return (params.slCalib.T[0] * params.f) / disp

uv = (750, 300)
disparity = disparity_image[uv[1], uv[0]] / 16
depth = calc_z(disparity)
print(f"disparity at {uv}: {disparity}, depth: {depth}")

## Solutions



[Here](https://colab.research.google.com/drive/1wQjYxPYeFu_P2ZyZaSt4Z4eEPlgzuxDi?usp=sharing) you will find this same notebook, but with solutions.